In [8]:
import sys, os
import numpy as np
from numpy import genfromtxt
import pandas as pd
from scipy import stats
from scipy import sparse
from scipy.cluster import hierarchy
from scipy.spatial import distance
from sklearn.preprocessing import OneHotEncoder

import networkx as nx
from collections import Counter

import urllib.request
import re, json, requests, itertools

import statsmodels.api as sm
import statsmodels.formula.api as smf


# Plotting packages
import matplotlib.pyplot as plt
plt.rcdefaults()
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cmx
import pylab

import bff
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

Autosaving every 30 seconds


In [9]:
def ttest_ind(x1, x2, equivar=False, alpha=0.05, printres=False):
    n1 = len(x1)
    M1 = np.mean(x1)
    s1 = np.std(x1, ddof=1)
    n2 = len(x2)
    M2 = np.mean(x2)
    s2 = np.std(x2, ddof=1)
    
    # t-test
    [t, p] = stats.ttest_ind(x1, x2, equal_var=equivar)
    # cohen's d
    dof = n1 + n2 - 2
    sp = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / dof)
    d = np.abs(M1 - M2) / sp
    # degrees of freedom
    df = (s1**2/n1 + s2**2/n2)**2 / ((s1**2/n1)**2/(n1-1) + (s2**2/n2)**2/(n2-1))
    # confidence intervals (M1 - M2) ± ts(M1 - M2)
    se = np.sqrt(sp**2/n1 + sp**2/n2)
    CI = (M1 - M2) + np.array([-1,1])*stats.t.ppf(1-alpha/2, df, loc=0, scale=1)*se

    res = (t, df, p, d, CI[0], CI[1])
    if printres:
        print("t = %.5f, df = %.5f, p = %.5f, d = %.5f, CI = (%.5f, %.5f)" % res)
    else:
        return res

In [47]:
# Load file (from same directory as the notebook)
dfs = pd.read_excel(os.path.expanduser("submetnew.xlsx"))
dfs.head()

,org,rank,org_size,av_clustering,av_hierarchy,av_eigcen,av_betcen,acad_ind,gender,topic,endwt,race
0,colorado,2.4,18,0.333333,0.044983,1.210000e-09,1.664120e-04,Academic,0.173913,2.351807,1.53,NaN
1,oregonstate,2.5,15,0.400000,0.061224,6.390000e-08,0.000000e+00,Academic,0.200000,2.026230,0.60,0.48
2,buffalo,2.7,23,0.613043,0.116736,2.490000e-06,4.620000e-08,Academic,0.333333,2.370044,0.70,NaN
3,indiana,2.9,18,0.659259,0.173010,2.430000e-09,3.920000e-05,Academic,0.111111,2.248386,NaN,NaN
4,rpi,2.9,17,0.176471,0.073568,6.150000e-10,1.650000e-08,Academic,0.100000,2.302759,0.70,NaN


In [38]:
from sklearn.preprocessing import StandardScaler

In [40]:
scaler = StandardScaler()

In [48]:
dfs[['av_clustering', 'av_hierarchy', 'av_betcen', 'gender', 'topic', 'av_eigcen']] = scaler.fit_transform(dfs[['av_clustering', 'av_hierarchy', 'av_betcen', 'gender', 'topic', 'av_eigcen']])
dfs.head()

,org,rank,org_size,av_clustering,av_hierarchy,av_eigcen,av_betcen,acad_ind,gender,topic,endwt,race
0,colorado,2.4,18,-0.819223,-1.385512,-0.146023,0.281761,Academic,0.175237,0.239541,1.53,NaN
1,oregonstate,2.5,15,-0.497101,-1.161712,-0.145986,-0.701751,Academic,0.332878,-0.595496,0.60,0.48
2,buffalo,2.7,23,0.532288,-0.396810,-0.144564,-0.701478,Academic,1.138601,0.286316,0.70,NaN
3,indiana,2.9,18,0.755595,0.378617,-0.146022,-0.470075,Academic,-0.204270,-0.025710,NaN,NaN
4,rpi,2.9,17,-1.577156,-0.991631,-0.146023,-0.701654,Academic,-0.271414,0.113745,0.70,NaN


In [34]:
import warnings
warnings.filterwarnings('ignore')

%load_ext rpy2.ipython
%R library(lmerTest)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


array(['lmerTest', 'lme4', 'Matrix', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], dtype='<U9')

In [49]:
%Rpush dfs

In [46]:
%%R

M <- lmer(av_betcen ~ av_clustering + av_hierarchy + gender + topic + + (1|acad_ind) + (1|org_size), data = dfs)
print(summary(M))

R[write to console]: boundary (singular) fit: see ?isSingular



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: av_betcen ~ av_clustering + av_hierarchy + gender + topic + +(1 |  
    acad_ind) + (1 | org_size)
   Data: dfs

REML criterion at convergence: 392.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.2421 -0.5479 -0.2070  0.3821  3.9091 

Random effects:
 Groups   Name        Variance Std.Dev.
 org_size (Intercept) 0.05203  0.2281  
 acad_ind (Intercept) 0.00000  0.0000  
 Residual             0.70908  0.8421  
Number of obs: 149, groups:  org_size, 63; acad_ind, 2

Fixed effects:
               Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)     0.01482    0.07897  30.29056   0.188   0.8524    
av_clustering  -0.14833    0.08256 142.61081  -1.797   0.0745 .  
av_hierarchy    0.39147    0.08124 142.35964   4.818 3.66e-06 ***
gender          0.13800    0.07072 131.20577   1.951   0.0531 .  
topic           0.39299    0.07590  50.28631   5.178 3.96e-06 ***
---
Signi

In [50]:
%%R

M <- lmer(av_eigcen ~ av_clustering + av_hierarchy + gender + topic + + (1|acad_ind) + (1|org_size), data = dfs)
print(summary(M))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: av_eigcen ~ av_clustering + av_hierarchy + gender + topic + +(1 |  
    acad_ind) + (1 | org_size)
   Data: dfs

REML criterion at convergence: 360.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.4803 -0.0847 -0.0364  0.0851  5.9077 

Random effects:
 Groups   Name        Variance Std.Dev.
 org_size (Intercept) 1.87435  1.3691  
 acad_ind (Intercept) 0.05152  0.2270  
 Residual             0.16272  0.4034  
Number of obs: 149, groups:  org_size, 63; acad_ind, 2

Fixed effects:
               Estimate Std. Error        df t value Pr(>|t|)
(Intercept)    0.204308   0.244914  3.965594   0.834    0.451
av_clustering  0.011491   0.048701 85.094499   0.236    0.814
av_hierarchy  -0.007125   0.046265 82.440666  -0.154    0.878
gender         0.006081   0.037336 80.457004   0.163    0.871
topic          0.076274   0.071088 98.609346   1.073    0.286

Correlation of Fixed Effects:
  